In [15]:
### Load packages
library('gbm')          # Gradient Boosting Machine
library('randomForest') # Random Forest
library('parallel')
library('caret')

### Load Boosted-RGB methods
source('./experiments/covariance/R/covFunctions.R')
source('./experiments/classification/R/rgbFunctions.R')
source('./experiments/classification/R/utilityFunctions.R')

### Instantiate cluster
cl <- parallel::makeCluster(4, type="SOCK") # Train BRGB using 4 cores
parallel::clusterExport(cl, ls())

### Choose dataset
load('./experiments/classification/data/data_car.RData')

### Split data
train.idx <- caret::createDataPartition(y, p = 0.7, list = FALSE)
x.tr <- x[train.idx,]
x.te <- x[-train.idx,]
y.tr <- y[train.idx]
y.te <- y[-train.idx]

### Train classifiers
brgb.model <- trainClassifier('BoostedRgb', x.tr, y.tr)
gbm.model  <- trainClassifier('GradientBoosting', x.tr, y.tr)
rf.model   <- trainClassifier('RandomForest', x.tr, y.tr)

### Test classifiers
brgb.resp <- testClassifier(brgb.model, x.te, y.te, 'car')
gbm.resp  <- testClassifier(gbm.model, x.te, y.te, 'car')
rf.resp   <- testClassifier(rf.model, x.te, y.te, 'car')

### Merge results
results <- do.call('rbind.data.frame', list(brgb.resp, gbm.resp, rf.resp))
rownames(results) <- NULL
colnames(results) <- c('Dataset', 'Algorithm', 'Accuracy', 'AUC', 'Log-Loss', 'Training Time')
print(results)

### Close cluster
parallel::stopCluster(cl)

  Dataset        Algorithm  Accuracy       AUC  Log-Loss Training Time
1     car       BoostedRgb 0.9787234 0.9976123 0.3932066         1.590
2     car GradientBoosting 0.9574468 0.9957008 0.1513304         1.045
3     car     RandomForest 0.8742747 0.9834811 0.3364408         0.305
